In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, RepeatedStratifiedKFold, LeaveOneOut, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score,classification_report, f1_score, roc_auc_score
import sklearn.metrics as metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler, OneHotEncoder, LabelEncoder, PowerTransformer, QuantileTransformer
from sklearn.pipeline import Pipeline, make_pipeline
import warnings
warnings.filterwarnings("ignore")
from statistics import mean
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import plotly.express as px
import pickle
%run auxiliar.py

# Definir la metrica a utilizar 


In [113]:
scorer = metrics.make_scorer(metrics.f1_score)## Modelado

In [114]:
# Lectura de los datos
data = pd.read_csv("../data/superstore_clean.csv")

In [115]:
df_limpio = data.copy()

In [116]:
list(data.columns.values)

['Education_Graduation',
 'Education_PhD',
 'Education_Master',
 'Education_Basic',
 'Marital_Status_Divorced',
 'Marital_Status_Single',
 'Marital_Status_Married',
 'Income',
 'Kidhome',
 'Teenhome',
 'Recency',
 'MntWines',
 'MntFruits',
 'MntMeatProducts',
 'MntFishProducts',
 'MntSweetProducts',
 'MntGoldProds',
 'NumDealsPurchases',
 'NumWebPurchases',
 'NumCatalogPurchases',
 'NumStorePurchases',
 'NumWebVisitsMonth',
 'Response',
 'Complain',
 'Edad',
 'Dt_Customer_year',
 'tiempo_participacion']

In [117]:
seed = 12345

In [118]:
# # Solo con columnas seleccionadas en proceso de seleccion de variables
predictors = df_limpio[['Recency', 'NumStorePurchases', 'NumCatalogPurchases', 'MntWines', 'MntGoldProds', 'Teenhome', 'MntMeatProducts', 'NumWebVisitsMonth', 'Marital_Status_Married', 'Income']]


In [119]:
# X = predictors
X = predictors
y = df_limpio.Response

In [120]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = seed, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1565, 10), (671, 10), (1565,), (671,))

In [121]:
%%time
# Definir modelo
cbc = CatBoostClassifier(logging_level = 'Silent', random_state = seed)

# Transformador
scaler = QuantileTransformer()

# Pipeline
pipeline = Pipeline([('Quantile', scaler),
                    ('Modelo', cbc)])

# Crear la grilla
grid = {'Modelo__max_depth': [3,4,5],'Modelo__n_estimators':[100, 200, 300],'Modelo__learning_rate' : [0.01, 0.05, 0.1]}

#cv
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

# Llama a grid search
gscv = GridSearchCV (estimator = pipeline, param_grid = grid, scoring = scorer, cv = cv, verbose = False)

#fit the model
gscv.fit(X_train,y_train)

# Resultados
print(f'Mejor estimador: {gscv.best_estimator_}')
print(f'Mejores parametros: {gscv.best_params_}')
print(f'Mejor score: {gscv.best_score_}')


Mejor estimador: Pipeline(steps=[('Quantile', QuantileTransformer()),
                ('Modelo',
                 <catboost.core.CatBoostClassifier object at 0x7f9830453cd0>)])
Mejores parametros: {'Modelo__learning_rate': 0.1, 'Modelo__max_depth': 5, 'Modelo__n_estimators': 200}
Mejor score: 0.485768967874231
CPU times: user 31.9 s, sys: 23.5 s, total: 55.5 s
Wall time: 22.7 s


In [122]:
catboost_t = CatBoostClassifier(
    max_depth = 5,
    n_estimators = 200,
    learning_rate = 0.1,
    scale_pos_weight = 3,
    random_state = seed,
    logging_level = 'Silent' #para que no salga print de iteraciones

)

catboost_t.fit(X_train, y_train,eval_set = (X_test, y_test), early_stopping_rounds = 30,use_best_model = True)

In [123]:
# Resultados en train
res_cat_3 = reportes(catboost_t, X_train, y_train, nombre = "cat train scale")
res_cat_3

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train scale,0.9054,0.7521,0.8423,0.6617,0.704


In [124]:
# Resultados en train
res_cat_3_t = reportes(catboost_t, X_test, y_test, nombre = "cat train scale")
res_cat_3_t

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train scale,0.8718,0.51,0.7226,0.5795,0.5426


In [97]:
# Guarda el modelo
pickle.dump(catboost_t, open('../web/model_10.pkl','wb'))